<a href="https://colab.research.google.com/github/Andfeesq/BIG-DATA-SEMINARY.github.io/blob/main/LABORATORIO_N%C2%B02_PANDAS_vs_POLAR_vs_SPARK_vs_DASK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pandas vs Polars vs Spark vs Dask

##LABORATORIO N°2 ANDRES FELIPE ESQUIVEL RUIZ, NATHALY DANIELA MEJIA MELENDEZ

TOMADO DE:

[Author: Elias Buitrago Bolivar](https://github.com/ebuitrago?tab=repositories)

Inspired in: https://www.youtube.com/watch?v=mi9f9zOaqM8

Original data: Kaggle

This jupyter notebook is designed to study and compare different tools to read and manipulate data; to be used in the data undertanding phase. The corresponding explanations will be given directly in class, therefore the material isn't autoexplained. Don´t forget ask me for the access to the data. And, please, give credits to the original author's idea and, if consider, also to me.

_Updated: June 20th, 2023_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Playing with pandas

In [ ]:
import pandas as pd
flights_file1 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2018 .parquet"
flights_file2 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2019.parquet"
#flights_file3 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2020.parquet"
#flights_file4 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2021.parquet"
# flights_file5 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2022.parquet"
df1 = pd.read_parquet(flights_file1)
df2 = pd.read_parquet(flights_file2)
# df3 = pd.read_parquet(flights_file3)
# df4 = pd.read_parquet(flights_file4)
# df5 = pd.read_parquet(flights_file5)

In [ ]:
# df = pd.concat([df3, df5])
df = df2

In [ ]:
# %%timeit

df_agg = df.groupby(['Airline','Year'])[["DepDelayMinutes", "ArrDelayMinutes"]].agg(
    ["mean", "sum", "max"]
)
df_agg = df_agg.reset_index()
df_agg.to_parquet("temp_pandas.parquet")

In [ ]:
!ls -GFlash temp_pandas.parquet

In [ ]:
pd.read_parquet('temp_pandas.parquet')

In [ ]:
pd.read_parquet('temp_pandas.parquet').info()

## Playing with Polars

In [ ]:
import polars as pl

In [ ]:
flights_file1 = "/content/drive/MyDrive/SEMINARIO BIG DATA/EJERCICIOS/CLASE 02.07.2024/Combined_Flights_2018 .parquet"
flights_file2 = "/content/drive/MyDrive/SEMINARIO BIG DATA/EJERCICIOS/CLASE 02.07.2024/Combined_Flights_2019.parquet"
flights_file3 = "/content/drive/MyDrive/SEMINARIO BIG DATA/EJERCICIOS/CLASE 02.07.2024/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/SEMINARIO BIG DATA/EJERCICIOS/CLASE 02.07.2024/Combined_Flights_2021.parquet"
flights_file5 = "/content/drive/MyDrive/SEMINARIO BIG DATA/EJERCICIOS/CLASE 02.07.2024/Combined_Flights_2022.parquet"
df1 = pl.scan_parquet(flights_file1)
df2 = pl.scan_parquet(flights_file2)
df3 = pl.scan_parquet(flights_file3)
df4 = pl.scan_parquet(flights_file4)
df5 = pl.scan_parquet(flights_file5)

In [ ]:
 %%timeit

df_polars = (
    pl.concat([df1, df2, df3, df4, df5])
    .groupby(['Airline', 'Year'])
    .agg([
        pl.col("DepDelayMinutes").mean().alias("avg_dep_delay"),
        pl.col("DepDelayMinutes").sum().alias("sum_dep_delay"),
        pl.col("DepDelayMinutes").max().alias("max_dep_delay"),
        pl.col("ArrDelayMinutes").mean().alias("avg_arr_delay"),
        pl.col("ArrDelayMinutes").sum().alias("sum_arr_delay"),
        pl.col("ArrDelayMinutes").max().alias("max_arr_delay"),
      ])
).collect()

df_polars.write_parquet('temp_polars.parquet')

<magic-timeit>:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.


10.7 s ± 879 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
!ls -GFlash temp_polars.parquet

12K -rw-r--r-- 1 root 8.1K Jul  3 00:29 temp_polars.parquet


## Playing with PySpark

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max, sum, concat

In [ ]:
spark = SparkSession.builder.master("local[1]").appName("airline-example").getOrCreate()

In [ ]:
flights_file1 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2018 .parquet"
flights_file2 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2019.parquet"
flights_file3 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2021.parquet"
flights_file5 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2022.parquet"

In [ ]:
df_spark1 = spark.read.parquet(flights_file1)
df_spark2 = spark.read.parquet(flights_file2)
df_spark3 = spark.read.parquet(flights_file3)
df_spark4 = spark.read.parquet(flights_file4)
df_spark5 = spark.read.parquet(flights_file5)

In [ ]:
df_spark = df_spark1.union(df_spark2)
df_spark = df_spark.union(df_spark3)
df_spark = df_spark.union(df_spark4)
df_spark = df_spark.union(df_spark5)

In [ ]:
 %%timeit

df_spark_agg = df_spark.groupby("Airline", "Year").agg(
    avg("ArrDelayMinutes").alias('avg_arr_delay'),
    sum("ArrDelayMinutes").alias('sum_arr_delay'),
    max("ArrDelayMinutes").alias('max_arr_delay'),
    avg("DepDelayMinutes").alias('avg_dep_delay'),
    sum("DepDelayMinutes").alias('sum_dep_delay'),
    max("DepDelayMinutes").alias('max_dep_delay'),
)
df_spark_agg.write.mode('overwrite').parquet('temp_spark.parquet')

10.1 s ± 743 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
!ls -GFlash temp_pyspark.parquet

total 24K
4.0K drwxr-xr-x 2 root 4.0K Jun 19 19:50 ./
4.0K drwxr-xr-x 1 root 4.0K Jun 19 20:08 ../
8.0K -rw-r--r-- 1 root 7.5K Jun 19 19:50 part-00000-d120adcf-cbd9-4950-a0e9-617fa7ca2c45-c000.snappy.parquet
4.0K -rw-r--r-- 1 root   68 Jun 19 19:50 .part-00000-d120adcf-cbd9-4950-a0e9-617fa7ca2c45-c000.snappy.parquet.crc
   0 -rw-r--r-- 1 root    0 Jun 19 19:50 _SUCCESS
4.0K -rw-r--r-- 1 root    8 Jun 19 19:50 ._SUCCESS.crc


## Playing with dask

In [ ]:
import pandas as pd
import dask.dataframe as dd
flights_file1 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2018 .parquet"
flights_file2 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2019.parquet"
flights_file3 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2020.parquet"
flights_file4 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2021.parquet"
flights_file5 = "/content/drive/MyDrive/SEMINARIO BIG DATA/PROFESOR ELIAS/EJERCICIOS/Pandas vs Polars vs Spark vs Dask/DATOS/Combined_Flights_2022.parquet"
df1 = dd.read_parquet(flights_file1)
df2 = dd.read_parquet(flights_file2)
df3 = dd.read_parquet(flights_file3)
df4 = dd.read_parquet(flights_file4)
df5 = dd.read_parquet(flights_file5)

In [ ]:
df = dd.concat([df3, df5])

In [ ]:
print(df.compute())

       FlightDate            Airline Origin Dest  Cancelled  Diverted  \
0      2020-09-01        Comair Inc.    PHL  DAY      False     False   
1      2020-09-02        Comair Inc.    PHL  DAY      False     False   
2      2020-09-03        Comair Inc.    PHL  DAY      False     False   
3      2020-09-04        Comair Inc.    PHL  DAY      False     False   
4      2020-09-05        Comair Inc.    PHL  DAY      False     False   
...           ...                ...    ...  ...        ...       ...   
590537 2022-03-31  Republic Airlines    MSY  EWR      False      True   
590538 2022-03-17  Republic Airlines    CLT  EWR       True     False   
590539 2022-03-08  Republic Airlines    ALB  ORD      False     False   
590540 2022-03-25  Republic Airlines    EWR  PIT      False      True   
590541 2022-03-07  Republic Airlines    EWR  RDU      False      True   

        CRSDepTime  DepTime  DepDelayMinutes  DepDelay  ...  WheelsOff  \
0             1905   1858.0              0.0     

In [ ]:
df = df.compute()

In [ ]:
# %%timeit

df_agg = df.groupby(['Airline','Year'])[["DepDelayMinutes", "ArrDelayMinutes"]].agg(
    ["mean", "sum", "max"]
)
df_agg = df_agg.reset_index()
df_agg.to_parquet("temp_dask.parquet")

In [ ]:
!ls -GFlash temp_pandas.parquet

In [ ]:
pd.read_parquet('temp_dask.parquet').info()

In [ ]:
pd.read_parquet('temp_dask.parquet')

## Read Results

In [ ]:
import pandas as pd

In [ ]:
agg_pandas = pd.read_parquet('temp_pandas.parquet')
agg_polars = pd.read_parquet('temp_polars.parquet')
agg_spark = pd.read_parquet('temp_spark.parquet')
agg_dask = pd.read_parquet('temp_dask.parquet')

In [ ]:
agg_pandas.shape, agg_polars.shape, agg_spark.shape, agg_dask.shape

((26, 8), (122, 8), (122, 8), (26, 8))

In [ ]:
agg_pandas.sort_values(['Airline','Year']).head()

Airline  Year DepDelayMinutes                      \
                                                mean         sum     max   
0  Air Wisconsin Airlines Corp  2019       16.868511   1742281.0  1690.0   
1         Alaska Airlines Inc.  2019        9.836041   2576246.0  1117.0   
2                Allegiant Air  2019       14.678433   1536876.0  1979.0   
3       American Airlines Inc.  2019       14.895515  13814816.0  2315.0   
4  Capital Cargo International  2019       11.525332   1367642.0  1182.0   

  ArrDelayMinutes                      
             mean         sum     max  
0       17.610384   1811545.0  1707.0  
1       10.787284   2815643.0  1087.0  
2       15.556524   1624770.0  1966.0  
3       15.251863  14096412.0  2350.0  
4       12.489465   1474806.0  1190.0

In [ ]:
agg_polars.sort_values(['Airline','Year']).head()

,Airline,Year,avg_dep_delay,sum_dep_delay,max_dep_delay,avg_arr_delay,sum_arr_delay,max_arr_delay
28,Air Wisconsin Airlines Corp,2018,16.753459,1606774.0,1296.0,17.881934,1708887.0,1292.0
54,Air Wisconsin Airlines Corp,2019,16.868511,1742281.0,1690.0,17.610384,1811545.0,1707.0
106,Air Wisconsin Airlines Corp,2020,8.583725,433315.0,1460.0,8.982529,452450.0,1439.0
6,Air Wisconsin Airlines Corp,2021,16.553045,1290194.0,1421.0,17.327440,1346602.0,1416.0
67,Air Wisconsin Airlines Corp,2022,13.124801,510581.0,1355.0,13.340409,517261.0,1353.0


In [ ]:
agg_spark.sort_values(['Airline','Year']).head()

,Airline,Year,avg_arr_delay,sum_arr_delay,max_arr_delay,avg_dep_delay,sum_dep_delay,max_dep_delay
0,Air Wisconsin Airlines Corp,2018,17.881934,1708887.0,1292.0,16.753459,1606774.0,1296.0
48,Air Wisconsin Airlines Corp,2019,17.610384,1811545.0,1707.0,16.868511,1742281.0,1690.0
56,Air Wisconsin Airlines Corp,2020,8.982529,452450.0,1439.0,8.583725,433315.0,1460.0
93,Air Wisconsin Airlines Corp,2021,17.327440,1346602.0,1416.0,16.553045,1290194.0,1421.0
119,Air Wisconsin Airlines Corp,2022,13.340409,517261.0,1353.0,13.124801,510581.0,1355.0


In [ ]:
agg_dask.sort_values(['Airline','Year']).head()

Airline  Year DepDelayMinutes                      \
                                                mean         sum     max   
0  Air Wisconsin Airlines Corp  2019       16.868511   1742281.0  1690.0   
1         Alaska Airlines Inc.  2019        9.836041   2576246.0  1117.0   
2                Allegiant Air  2019       14.678433   1536876.0  1979.0   
3       American Airlines Inc.  2019       14.895515  13814816.0  2315.0   
4  Capital Cargo International  2019       11.525332   1367642.0  1182.0   

  ArrDelayMinutes                      
             mean         sum     max  
0       17.610384   1811545.0  1707.0  
1       10.787284   2815643.0  1087.0  
2       15.556524   1624770.0  1966.0  
3       15.251863  14096412.0  2350.0  
4       12.489465   1474806.0  1190.0